In [16]:
%load_ext autoreload
%autoreload 2

from experiments.all import train_and_evaluate
from tsm.configuration import TSMConfigParser
import docker
import logging

logging.basicConfig(level=logging.INFO)

# load configuration
config = TSMConfigParser()
config.read("/home/pablo/tesis/tsm-pipeline/code/configs/pablo-bestia.cfg")
config.check()
docker_client = docker.from_env()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
from scripts.docker import read_dbs_dataset
from numpy.random import RandomState

sql_dbs = read_dbs_dataset("/home/pablo/tesis/tsm-pipeline/experiments/datasets/sql_1.txt")
# Using a fixed random state to have reproducible results
rand = RandomState(seed=5)

print(len(sql_dbs))

42


Testing the sql module works

In [5]:
train_sample = rand.choice(sql_dbs, size=5)
test_sample = rand.choice(sql_dbs, size=3)

In [ ]:
scores = train_and_evaluate(
            config=config,
            results_dir="/home/pablo/dev-results/sql-test",
            query_type="sql",
            train=train_sample.tolist(),
            test=test_sample.tolist(),
            docker_client=docker_client,
            skip=["train"] # skip training since it has already ran
        )
scores

## Corrida para tesis

In [11]:
from sklearn.model_selection import KFold
# fixing random state to make this repeatable
kf = KFold(n_splits=5)
folds = list(kf.split(sql_dbs))

In [20]:
for i, fold in enumerate(folds):
    print("fold %d: TRAIN %d - TEST %d" % (i, len(fold[0]), len(fold[1])))
    print(fold)

fold 0: TRAIN 33 - TEST 9
(array([ 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25,
       26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]), array([0, 1, 2, 3, 4, 5, 6, 7, 8]))
fold 1: TRAIN 33 - TEST 9
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 18, 19, 20, 21, 22, 23, 24, 25,
       26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]), array([ 9, 10, 11, 12, 13, 14, 15, 16, 17]))
fold 2: TRAIN 34 - TEST 8
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]), array([18, 19, 20, 21, 22, 23, 24, 25]))
fold 3: TRAIN 34 - TEST 8
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 34, 35, 36, 37, 38, 39, 40, 41]), array([26, 27, 28, 29, 30, 31, 32, 33]))
fold 4: TRAIN 34 - TEST 8
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 

In [14]:
import numpy as np
base_results_dir = "/home/pablo/dev-results/sql-5fold-"
for i, (train_index, test_index) in enumerate(folds):
    try:
        print("Fold %d" % (i))
        res_dir = "%s%d" % (base_results_dir, i)
        train_split = np.take(sql_dbs, train_index)
        test_split = np.take(sql_dbs, test_index)

        scores = train_and_evaluate(
            config=config,
            results_dir=res_dir,
            query_type="sql",
            train=train_split.tolist(),
            test=test_split.tolist(),
            docker_client=docker_client,
        )
        print(scores)
    except Exception as e:
        print(f'fold {i} failed fatally. Please retry!')
        print(e)
        continue
print("done")

[INFO	2023-07-23 14:52:09,306] experiments.all	train_and_evaluate with train size 33, test size 9
[INFO	2023-07-23 14:52:09,307] experiments.all	Running training
[INFO	2023-07-23 14:52:09,307] scripts.docker	creating /home/pablo/dev-results/sql-5fold-0 dir. Omitting error if already existing
[INFO	2023-07-23 14:52:09,307] scripts.docker	creating /tmp/log dir. Omitting error if already existing


Fold 0


[INFO	2023-07-23 14:52:09,586] scripts.docker	running at container b9a91c0330698bcc950484d07d45657ecb53f505d2eb4a563c7274b2c639e950. Use `docker logs b9a91c0330698bcc950484d07d45657ecb53f505d2eb4a563c7274b2c639e950 --tail 10 --follow` to follow progress
[INFO	2023-07-23 15:00:18,753] scripts.docker	running combine scores
[INFO	2023-07-23 15:00:18,754] misc.combinescores	globbing folder: /home/pablo/dev-results/sql-5fold-0/*/reprScores.txt
[INFO	2023-07-23 15:00:18,755] misc.combinescores	working on 22 reprScores files
[INFO	2023-07-23 15:00:18,759] experiments.all	Running worse evaluation
[INFO	2023-07-23 15:00:18,759] scripts.evaluate	creating /home/pablo/dev-results/sql-5fold-0/worse dir. Omitting error if already existing
[INFO	2023-07-23 15:00:18,938] scripts.evaluate	Starting processing with 4 processes
[INFO	2023-07-23 15:00:18,959] scripts.evaluate	Evaluating TheKingdomDev - Burger
[INFO	2023-07-23 15:00:18,959] scripts.evaluate	Evaluating OWASP - samm
[INFO	2023-07-23 15:00:18,

   precision  recall  accuracy  alerts to recover (atr)  alerts recovered  \
0        0.0     0.0       0.0                    13599                 0   

   suprious alerts  projects with atr  avg atr per proj  
0                2                  8          1699.875  
Fold 1


[INFO	2023-07-23 15:01:02,710] scripts.docker	running at container 1646757a7b1e490ba232be8898c4a7ae04da08ee4a17ceba3428bac2e0e96e40. Use `docker logs 1646757a7b1e490ba232be8898c4a7ae04da08ee4a17ceba3428bac2e0e96e40 --tail 10 --follow` to follow progress
[INFO	2023-07-23 15:07:39,007] scripts.docker	running combine scores
[INFO	2023-07-23 15:07:39,007] misc.combinescores	globbing folder: /home/pablo/dev-results/sql-5fold-1/*/reprScores.txt
[INFO	2023-07-23 15:07:39,008] misc.combinescores	working on 26 reprScores files
[INFO	2023-07-23 15:07:39,012] experiments.all	Running worse evaluation
[INFO	2023-07-23 15:07:39,012] scripts.evaluate	creating /home/pablo/dev-results/sql-5fold-1/worse dir. Omitting error if already existing
[INFO	2023-07-23 15:07:39,194] scripts.evaluate	Starting processing with 4 processes
[INFO	2023-07-23 15:07:39,211] scripts.evaluate	Evaluating cbrannon - burger
[INFO	2023-07-23 15:07:39,210] scripts.evaluate	Evaluating arukpupatee - what2eat
[INFO	2023-07-23 15:0

...

   precision  recall  accuracy  alerts to recover (atr)  alerts recovered  \
0        0.0     0.0       0.0                       21                 0   

   suprious alerts  projects with atr  avg atr per proj  
0               54                  5               4.2  
Fold 3


[INFO	2023-07-23 15:15:59,022] scripts.docker	running at container 82beb1b0eb5441fd835347f03fac7ff4575d858f0cfc63f10ecf4b358effdd1b. Use `docker logs 82beb1b0eb5441fd835347f03fac7ff4575d858f0cfc63f10ecf4b358effdd1b --tail 10 --follow` to follow progress
[INFO	2023-07-23 15:22:48,674] scripts.docker	running combine scores
[INFO	2023-07-23 15:22:48,674] misc.combinescores	globbing folder: /home/pablo/dev-results/sql-5fold-3/*/reprScores.txt
[INFO	2023-07-23 15:22:48,675] misc.combinescores	working on 25 reprScores files
[INFO	2023-07-23 15:22:48,680] experiments.all	Running worse evaluation
[INFO	2023-07-23 15:22:48,680] scripts.evaluate	creating /home/pablo/dev-results/sql-5fold-3/worse dir. Omitting error if already existing
[INFO	2023-07-23 15:22:48,859] scripts.evaluate	Starting processing with 4 processes
[INFO	2023-07-23 15:22:48,880] scripts.evaluate	Evaluating kwuellner - Burger
[INFO	2023-07-23 15:22:48,880] scripts.evaluate	Evaluating mheadd - civic-responsibility-api
[INFO	202

   precision  recall  accuracy  alerts to recover (atr)  alerts recovered  \
0        0.0     0.0       0.0                        5                 0   

   suprious alerts  projects with atr  avg atr per proj  
0                4                  3          1.666667  
Fold 4


[INFO	2023-07-23 15:23:23,335] scripts.docker	running at container 0f6e5aa0892d8f63c3e7a3a1a06dbec72f8541cd26b4a8b1052388116183ca8f. Use `docker logs 0f6e5aa0892d8f63c3e7a3a1a06dbec72f8541cd26b4a8b1052388116183ca8f --tail 10 --follow` to follow progress
[INFO	2023-07-23 15:29:10,731] scripts.docker	running combine scores
[INFO	2023-07-23 15:29:10,732] misc.combinescores	globbing folder: /home/pablo/dev-results/sql-5fold-4/*/reprScores.txt
[INFO	2023-07-23 15:29:10,732] misc.combinescores	working on 26 reprScores files
[INFO	2023-07-23 15:29:10,737] experiments.all	Running worse evaluation
[INFO	2023-07-23 15:29:10,737] scripts.evaluate	creating /home/pablo/dev-results/sql-5fold-4/worse dir. Omitting error if already existing
[INFO	2023-07-23 15:29:10,914] scripts.evaluate	Starting processing with 4 processes
[INFO	2023-07-23 15:29:10,936] scripts.evaluate	Evaluating vulhub - vulhub
[INFO	2023-07-23 15:29:10,936] scripts.evaluate	Evaluating startergate - EnglishPresentation
[INFO	2023-0

   precision  recall  accuracy  alerts to recover (atr)  alerts recovered  \
0        0.0     0.0       0.0                        5                 0   

   suprious alerts  projects with atr  avg atr per proj  
0                2                  2               2.5  
done


In [15]:
import pandas as pd
from scripts.calculate_scores import calculate_scores_df

collected_results = []
for i in range(5):
    df = calculate_scores_df("%s%d" % (base_results_dir, i), cleanup_base_dir="tmp")
    collected_results.append(df)

concated_results = pd.concat(collected_results)
concated_results = pd.concat([concated_results, concated_results.apply(['mean'])])
concated_results['name'] = [f'fold {i}' for i in range(5)] + ['mean']
concated_results

,precision,recall,accuracy,alerts to recover (atr),alerts recovered,suprious alerts,projects with atr,avg atr per proj,name
0,0.0,0.0,0.0,13599.0,0.0,2.0,8.0,1699.875000,fold 0
0,0.0,0.0,0.0,22.0,0.0,2.0,7.0,3.142857,fold 1
0,0.0,0.0,0.0,21.0,0.0,54.0,5.0,4.200000,fold 2
0,0.0,0.0,0.0,5.0,0.0,4.0,3.0,1.666667,fold 3
0,0.0,0.0,0.0,5.0,0.0,2.0,2.0,2.500000,fold 4
mean,0.0,0.0,0.0,2730.4,0.0,12.8,5.0,342.276905,mean


como la de arriba dio mal, voy a hacer 5 random splits de 50 / 50. Los set de arriba solo utilizanun set de evaluación de 9 dbs, que es muy poco. Para ello, haremos un ShuffleSplit de 50/50, 5 veces.

[Ref](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html)

In [23]:
from sklearn.model_selection import ShuffleSplit
rand_seed = 42
shuffler = ShuffleSplit(n_splits=5, test_size=.5, random_state=rand_seed)
splits = shuffler.split(sql_dbs)
splits = list(splits)
splits

In [26]:
for i, fold in enumerate(splits):
    print("fold %d: TRAIN %d - TEST %d" % (i, len(fold[0]), len(fold[1])))
    print(fold)

fold 0: TRAIN 21 - TEST 21
(array([ 5, 11, 40,  1, 31, 21,  2, 32, 41,  3, 37, 23, 34, 10, 22, 18, 20,
        7, 14, 28, 38]), array([25, 13,  8, 26,  4, 39, 19, 29, 30,  6, 27, 12, 17,  9, 15, 33, 16,
       36, 35,  0, 24]))
fold 1: TRAIN 21 - TEST 21
(array([40, 16, 36,  2, 30,  7, 11,  6, 31, 14, 27, 19,  1, 25, 41, 13, 24,
        3, 17, 38,  8]), array([ 4, 37, 10,  0, 35, 32, 28, 12, 23, 29, 34, 20, 15, 18,  9, 22,  5,
       33, 21, 26, 39]))
fold 2: TRAIN 21 - TEST 21
(array([26, 16, 18, 10, 29,  7, 11, 33, 38,  8,  6, 36,  0, 14, 28, 24, 25,
       23, 17, 15, 20]), array([ 3, 34, 32, 19, 39,  5, 12, 37,  1, 21, 13, 40, 27, 30, 22, 31,  9,
        4, 35, 41,  2]))
fold 3: TRAIN 21 - TEST 21
(array([16, 31, 18, 12,  9, 33, 28, 14,  8, 26,  6, 37, 25,  4, 40,  2, 13,
       36, 41,  0, 34]), array([ 7, 30, 35, 29, 23, 11,  1, 32,  5, 38, 15, 17, 20, 21, 10, 27, 24,
       22, 39,  3, 19]))
fold 4: TRAIN 21 - TEST 21
(array([ 7, 41,  6,  2,  8, 12, 38, 31, 37, 26, 34,  0, 32, 2

In [27]:
import numpy as np
base_results_dir = "/home/pablo/dev-results/sql-42-rand5fold-"
for i, (train_index, test_index) in enumerate(splits):
    try:
        print("Fold %d" % (i))
        res_dir = "%s%d" % (base_results_dir, i)
        train_split = np.take(sql_dbs, train_index)
        test_split = np.take(sql_dbs, test_index)

        scores = train_and_evaluate(
            config=config,
            results_dir=res_dir,
            query_type="sql",
            train=train_split.tolist(),
            test=test_split.tolist(),
            docker_client=docker_client,
        )
        print(scores)
    except Exception as e:
        print(f'fold {i} failed fatally. Please retry!')
        print(e)
        continue
print("done")

[INFO	2023-07-28 11:30:50,199] experiments.all	train_and_evaluate with train size 21, test size 21
[INFO	2023-07-28 11:30:50,200] experiments.all	Running training
[INFO	2023-07-28 11:30:50,200] scripts.docker	creating /home/pablo/dev-results/sql-42-rand5fold-0 dir. Omitting error if already existing
[INFO	2023-07-28 11:30:50,201] scripts.docker	creating /tmp/log dir. Omitting error if already existing


Fold 0


[INFO	2023-07-28 11:30:50,489] scripts.docker	running at container f123a3bb6e07988fd7d7fb2bfe22e8489aee5b0006057a5088d671fb3816491b. Use `docker logs f123a3bb6e07988fd7d7fb2bfe22e8489aee5b0006057a5088d671fb3816491b --tail 10 --follow` to follow progress
[INFO	2023-07-28 11:35:17,597] scripts.docker	running combine scores
[INFO	2023-07-28 11:35:17,597] misc.combinescores	globbing folder: /home/pablo/dev-results/sql-42-rand5fold-0/*/reprScores.txt
[INFO	2023-07-28 11:35:17,598] misc.combinescores	working on 16 reprScores files
[INFO	2023-07-28 11:35:17,601] experiments.all	Running worse evaluation
[INFO	2023-07-28 11:35:17,601] scripts.evaluate	creating /home/pablo/dev-results/sql-42-rand5fold-0/worse dir. Omitting error if already existing
[INFO	2023-07-28 11:35:17,798] scripts.evaluate	Starting processing with 4 processes
[INFO	2023-07-28 11:35:17,817] scripts.evaluate	Evaluating jordanbertasso - vulnerable-web-app
[INFO	2023-07-28 11:35:17,818] scripts.evaluate	Evaluating onap - ccsdk

   precision  recall  accuracy  alerts to recover (atr)  alerts recovered  \
0        0.0     0.0       0.0                       23                 0   

   suprious alerts  projects with atr  avg atr per proj  
0                6                 13          1.769231  
Fold 1


[INFO	2023-07-28 11:36:21,742] scripts.docker	running at container e7e5feaa578ddf00e2955c9299ee8cfa539e2e8a1a064a8121d20ba5319b1e71. Use `docker logs e7e5feaa578ddf00e2955c9299ee8cfa539e2e8a1a064a8121d20ba5319b1e71 --tail 10 --follow` to follow progress
[INFO	2023-07-28 11:40:45,443] scripts.docker	running combine scores
[INFO	2023-07-28 11:40:45,443] misc.combinescores	globbing folder: /home/pablo/dev-results/sql-42-rand5fold-1/*/reprScores.txt
[INFO	2023-07-28 11:40:45,444] misc.combinescores	working on 17 reprScores files
[INFO	2023-07-28 11:40:45,446] experiments.all	Running worse evaluation
[INFO	2023-07-28 11:40:45,446] scripts.evaluate	creating /home/pablo/dev-results/sql-42-rand5fold-1/worse dir. Omitting error if already existing
[INFO	2023-07-28 11:40:45,629] scripts.evaluate	Starting processing with 4 processes
[INFO	2023-07-28 11:40:45,648] scripts.evaluate	Evaluating QBisConsult - psql-api
[INFO	2023-07-28 11:40:45,648] scripts.evaluate	Evaluating typcn - Blogile
[INFO	202

...

   precision  recall  accuracy  alerts to recover (atr)  alerts recovered  \
0        0.0     0.0       0.0                    13615                 0   

   suprious alerts  projects with atr  avg atr per proj  
0                6                 13       1047.307692  
Fold 3


[INFO	2023-07-28 11:46:48,493] scripts.docker	running at container 6dee00a12cb6b9fe4fee430f3025bf18dbb71a579a59fa7f2a5c143bec17aa5a. Use `docker logs 6dee00a12cb6b9fe4fee430f3025bf18dbb71a579a59fa7f2a5c143bec17aa5a --tail 10 --follow` to follow progress
[INFO	2023-07-28 11:51:12,745] scripts.docker	running combine scores
[INFO	2023-07-28 11:51:12,746] misc.combinescores	globbing folder: /home/pablo/dev-results/sql-42-rand5fold-3/*/reprScores.txt
[INFO	2023-07-28 11:51:12,746] misc.combinescores	working on 16 reprScores files
[INFO	2023-07-28 11:51:12,748] experiments.all	Running worse evaluation
[INFO	2023-07-28 11:51:12,748] scripts.evaluate	creating /home/pablo/dev-results/sql-42-rand5fold-3/worse dir. Omitting error if already existing
[INFO	2023-07-28 11:51:12,948] scripts.evaluate	Starting processing with 4 processes
[INFO	2023-07-28 11:51:12,968] scripts.evaluate	Evaluating onap - ccsdk-distribution
[INFO	2023-07-28 11:51:12,968] scripts.evaluate	Evaluating antwan024 - burger
[IN

   precision  recall  accuracy  alerts to recover (atr)  alerts recovered  \
0        0.0     0.0       0.0                    13619                 0   

   suprious alerts  projects with atr  avg atr per proj  
0               56                 13       1047.615385  
Fold 4


[INFO	2023-07-28 11:52:22,714] scripts.docker	running at container b6e7f0b51a3c7d3b188b8a7f13bc9d2117c485e12685b618d31bfc93bf0b24ee. Use `docker logs b6e7f0b51a3c7d3b188b8a7f13bc9d2117c485e12685b618d31bfc93bf0b24ee --tail 10 --follow` to follow progress
[INFO	2023-07-28 11:57:26,101] scripts.docker	running combine scores
[INFO	2023-07-28 11:57:26,102] misc.combinescores	globbing folder: /home/pablo/dev-results/sql-42-rand5fold-4/*/reprScores.txt
[INFO	2023-07-28 11:57:26,103] misc.combinescores	working on 18 reprScores files
[INFO	2023-07-28 11:57:26,107] experiments.all	Running worse evaluation
[INFO	2023-07-28 11:57:26,107] scripts.evaluate	creating /home/pablo/dev-results/sql-42-rand5fold-4/worse dir. Omitting error if already existing
[INFO	2023-07-28 11:57:26,294] scripts.evaluate	Starting processing with 4 processes
[INFO	2023-07-28 11:57:26,320] scripts.evaluate	Evaluating hussien89aa - AndroidTutorialForBeginners
[INFO	2023-07-28 11:57:26,320] scripts.evaluate	Evaluating LMBoyl

   precision  recall  accuracy  alerts to recover (atr)  alerts recovered  \
0        0.0     0.0       0.0                    13634                 0   

   suprious alerts  projects with atr  avg atr per proj  
0               56                 14        973.857143  
done


In [28]:
from scripts.calculate_scores import calculate_many_scores

df = calculate_many_scores(["/home/pablo/dev-results/sql-42-rand5fold-%d" % i for i, _ in enumerate(splits)])
df

,precision,recall,accuracy,atr,recovered,spurious,progwithatr,avgatr,predsinks,name
0,0.0,0.0,0.0,23,0,6,13,1.769231,41,fold 0
0,0.0,0.0,0.0,33,0,29,10,3.300000,28,fold 1
0,0.0,0.0,0.0,13615,0,6,13,1047.307692,31,fold 2
0,0.0,0.0,0.0,13619,0,56,13,1047.615385,42,fold 3
0,0.0,0.0,0.0,13634,0,56,14,973.857143,57,fold 4
mean,0.0,0.0,0.0,8184,0,30,12,614.769890,39,mean


In [31]:
s = set(splits[0][1])
for (_, test) in splits:
    s = s.intersection(test)
sql_dbs[35]

'startergate/EnglishPresentation/3f83b65'

In [37]:
# prueba de entrenar y evaluar sobre el mismo set
train_split,_ = splits[1]
train = np.take(sql_dbs, train_split)

scores = train_and_evaluate(
            config=config,
            results_dir="/tmp/xss-prueba-overfitting",
            query_type="sql",
            train=train,
            test=train,
            docker_client=docker_client,
        )
scores

[INFO	2023-07-28 14:06:10,561] experiments.all	train_and_evaluate with train size 21, test size 21
[INFO	2023-07-28 14:06:10,562] experiments.all	Running training
[INFO	2023-07-28 14:06:10,562] scripts.docker	creating /tmp/xss-prueba-overfitting dir. Omitting error if already existing
[INFO	2023-07-28 14:06:10,563] scripts.docker	creating /tmp/log dir. Omitting error if already existing
[INFO	2023-07-28 14:06:10,820] scripts.docker	running at container 97790f3b3569062f3db6eeda86c1cf835c1253c2330c6fb0c654af67b8d549d1. Use `docker logs 97790f3b3569062f3db6eeda86c1cf835c1253c2330c6fb0c654af67b8d549d1 --tail 10 --follow` to follow progress
[INFO	2023-07-28 14:10:42,843] scripts.docker	running combine scores
[INFO	2023-07-28 14:10:42,844] misc.combinescores	globbing folder: /tmp/xss-prueba-overfitting/*/reprScores.txt
[INFO	2023-07-28 14:10:42,845] misc.combinescores	working on 17 reprScores files
[INFO	2023-07-28 14:10:42,847] experiments.all	Running worse evaluation
[INFO	2023-07-28 14:10

,precision,recall,accuracy,alerts to recover (atr),alerts recovered,suprious alerts,projects with atr,avg atr per proj
0,0.0,0.0,0.0,13619,0,4,15,907.933333


Probando allTargetLibraries en Propgraph-Sql

In [38]:
# prueba de entrenar y evaluar sobre el mismo set
train_split,_ = splits[1]
train = np.take(sql_dbs, train_split)

scores = train_and_evaluate(
            config=config,
            results_dir="/tmp/xss-prueba-overfitting-2",
            query_type="sql",
            train=train,
            test=train,
            docker_client=docker_client,
        )
scores

[INFO	2023-07-28 19:23:51,868] experiments.all	train_and_evaluate with train size 21, test size 21
[INFO	2023-07-28 19:23:51,876] experiments.all	Running training
[INFO	2023-07-28 19:23:51,877] scripts.docker	creating /tmp/xss-prueba-overfitting-2 dir. Omitting error if already existing
[INFO	2023-07-28 19:23:51,878] scripts.docker	creating /tmp/log dir. Omitting error if already existing
[INFO	2023-07-28 19:23:52,769] scripts.docker	running at container 6dd841caf82057dec7c6477ca5fb061080a7522a6816711af9528f056a522998. Use `docker logs 6dd841caf82057dec7c6477ca5fb061080a7522a6816711af9528f056a522998 --tail 10 --follow` to follow progress
[INFO	2023-07-28 19:28:13,029] scripts.docker	running combine scores
[INFO	2023-07-28 19:28:13,030] misc.combinescores	globbing folder: /tmp/xss-prueba-overfitting-2/*/reprScores.txt
[INFO	2023-07-28 19:28:13,030] misc.combinescores	working on 17 reprScores files
[INFO	2023-07-28 19:28:13,032] experiments.all	Running worse evaluation
[INFO	2023-07-28 1

,precision,recall,accuracy,alerts to recover (atr),alerts recovered,suprious alerts,projects with atr,avg atr per proj
0,0.0,0.0,0.0,13619,0,5,15,907.933333


In [39]:
from scripts.calculate_scores import calculate_many_scores
calculate_many_scores(["/tmp/xss-prueba-overfitting-2"])

,precision,recall,accuracy,atr,recovered,spurious,progwithatr,avgatr,predsinks,name
0,0.0,0.0,0.0,13619,0,5,15,907.933333,51,fold 0
mean,0.0,0.0,0.0,13619,0,5,15,907.933333,51,mean
